Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
import winsound

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'D:/Libraries/Documents/Tensorflow/notMNIST_data/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (300000, 28, 28) (300000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (300000, 784) (300000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [27]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / labels.shape[0])

In [5]:
def randomize(dataset, labels):
  permutation = np.random.permutation(labels.shape[0])
  shuffled_dataset = dataset[permutation,:]
  shuffled_labels = labels[permutation]
  return shuffled_dataset, shuffled_labels
train_dataset, train_labels = randomize(train_dataset, train_labels)
test_dataset, test_labels = randomize(test_dataset, test_labels)
valid_dataset, valid_labels = randomize(valid_dataset, valid_labels)

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [28]:
batch_size = 128
reg_constant = .002

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)
  )+reg_constant*tf.nn.l2_loss(t=weights)
  
  # Optimizer
  optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [29]:
num_steps = 3001
best_valid_acc = 0.0

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          valid_eval = accuracy(valid_prediction.eval(), valid_labels)
          print("Validation accuracy: %.1f%%" % valid_eval)
          if valid_eval < best_valid_acc:
                if(valid_eval < best_valid_acc - 2):
                    break;
          else:
                best_valid_acc = valid_eval
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
  single_layer_weights = weights.eval()

Initialized
Minibatch loss at step 0: 25.487446
Minibatch accuracy: 5.5%
Validation accuracy: 8.0%
Minibatch loss at step 500: 6.252701
Minibatch accuracy: 74.2%
Validation accuracy: 70.8%
Minibatch loss at step 1000: 5.261687
Minibatch accuracy: 68.8%
Validation accuracy: 74.0%
Minibatch loss at step 1500: 4.420434
Minibatch accuracy: 73.4%
Validation accuracy: 75.1%
Minibatch loss at step 2000: 3.089041
Minibatch accuracy: 85.2%
Validation accuracy: 76.2%
Minibatch loss at step 2500: 2.575495
Minibatch accuracy: 78.9%
Validation accuracy: 77.2%
Minibatch loss at step 3000: 2.052206
Minibatch accuracy: 82.0%
Validation accuracy: 77.7%
Test accuracy: 85.2%


Now the neural-network model

In [8]:
batch_size = 128
num_hidden_nodes = 2048
learning_rate=.01
reg_constant = .05

graph = tf.Graph()
with graph.as_default():
    
    #Constant Testing and Validation Data
    tf_test_data = tf.constant(test_dataset)
    tf_valid_data = tf.constant(valid_dataset)
    
    #Batch input training data for stochastic gradient descent
    tf_training_data = tf.placeholder(dtype=tf.float32,shape=(batch_size, image_size*image_size))
    tf_training_labels = tf.placeholder(dtype=tf.float32,shape=(batch_size,num_labels))
    
    #Variable weights to be optimized. Two layers
    weights1 = tf.Variable(tf.truncated_normal([image_size*image_size,num_hidden_nodes]))
    weights2 = tf.Variable(tf.truncated_normal([num_hidden_nodes,num_labels]))
    biases1 = tf.Variable(tf.zeros(num_hidden_nodes))
    biases2 = tf.Variable(tf.zeros(num_labels))
    
    
    #Creating graph structure
    hidden_layer_out = tf.nn.relu(tf.matmul(tf_training_data, weights1)+biases1)
    logits = tf.matmul(hidden_layer_out,weights2)+biases2
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=tf_training_labels)
    )+ reg_constant*tf.nn.l2_loss(t=weights1)+reg_constant*tf.nn.l2_loss(weights2)
    
    #Optimizer
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss=loss)
    
    #Output predictions
    tf_train_preds = tf.nn.softmax(logits)
    tf_test_preds = tf.nn.softmax(tf.matmul(tf.nn.relu(
        tf.matmul(tf_test_data,weights1)+biases1),weights2)+biases2)
    tf_valid_preds = tf.nn.softmax(tf.matmul(tf.nn.relu(
        tf.matmul(tf_valid_data,weights1)+biases1),weights2)+biases2)
    




In [9]:
num_steps = 3001
best_valid_acc = 0.0

with tf.Session(graph=graph) as session:
    
    tf.global_variables_initializer().run()
    
    for step in range(num_steps):
        #not the best randomization once you have >1 epoch, but works
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        #generate batches
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :].squeeze()
        
        #create feed-dictionary
        feed_dict = {tf_training_data: batch_data, tf_training_labels: batch_labels}
        
        #run session
        _, l, predictions = session.run([optimizer, loss, tf_train_preds], feed_dict=feed_dict)
        if (step % 1000 == 0):
            valid_eval = accuracy(tf_valid_preds.eval(), valid_labels)
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % valid_eval)
            if valid_eval < best_valid_acc:
                if(valid_eval < best_valid_acc-1):
                    break;
            else:
                best_valid_acc = valid_eval
          
    print("Test accuracy: %.1f%%" % accuracy(tf_test_preds.eval(), test_labels))
    winsound.Beep(500,1000)
        


Minibatch loss at step 0: 31931.083984
Minibatch accuracy: 9.4%
Validation accuracy: 12.7%
Minibatch loss at step 1000: 11572.437500
Minibatch accuracy: 78.9%
Validation accuracy: 78.6%
Minibatch loss at step 2000: 4249.183105
Minibatch accuracy: 82.0%
Validation accuracy: 80.0%
Minibatch loss at step 3000: 1561.216797
Minibatch accuracy: 85.9%
Validation accuracy: 81.2%
Test accuracy: 88.6%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. The training accuracy increases drastically, while the validation and test accuracies fall

---

In [10]:
batch_size = 128
reg_constant = .002

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)
  )+reg_constant*tf.nn.l2_loss(t=weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [11]:
num_steps = 3001
best_valid_acc = 0.0

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (1000)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          valid_eval = accuracy(valid_prediction.eval(), valid_labels)
          print("Validation accuracy: %.1f%%" % valid_eval)
          if valid_eval < best_valid_acc:
                if(valid_eval < best_valid_acc - 2):
                    break;
          else:
                best_valid_acc = valid_eval
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
  single_layer_weights = weights.eval()

Initialized
Minibatch loss at step 0: 21.511654
Minibatch accuracy: 8.6%
Validation accuracy: 10.8%
Minibatch loss at step 500: 2.472050
Minibatch accuracy: 92.2%
Validation accuracy: 71.7%
Minibatch loss at step 1000: 1.049329
Minibatch accuracy: 94.5%
Validation accuracy: 73.7%
Minibatch loss at step 1500: 0.564006
Minibatch accuracy: 96.9%
Validation accuracy: 75.2%
Minibatch loss at step 2000: 0.397564
Minibatch accuracy: 96.9%
Validation accuracy: 75.9%
Minibatch loss at step 2500: 0.339367
Minibatch accuracy: 96.9%
Validation accuracy: 76.3%
Minibatch loss at step 3000: 0.318734
Minibatch accuracy: 96.9%
Validation accuracy: 76.5%
Test accuracy: 84.3%


Changing to using the premade layers from tensorflow.

In [30]:
batch_size = 128
reg_constant = .002
hidden_nodes = 1024
learning_rate = 0.5

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_X_input = tf.placeholder(tf.float32,
                                    shape=(None, image_size * image_size))
  tf_y_input = tf.placeholder(tf.float32, shape=(None, num_labels))
  
  # Variables.
  tf_layer_1 = tf.layers.dense(tf_X_input,hidden_nodes)
  tf_layer_1 = tf.nn.relu(tf_layer_1)
  tf_layer_2 = tf.layers.dense(tf_layer_1,num_labels)

  # Training computation.
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_y_input, logits=tf_layer_2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  
  # Predictions
  tf_predictions = tf.nn.softmax(tf_layer_2)

In [42]:
num_steps = 5001
best_valid_acc = 0.0

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_X_input : batch_data, tf_y_input : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, tf_predictions], feed_dict=feed_dict)
    if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          valid_preds = session.run([tf_predictions], feed_dict={tf_X_input:valid_dataset})
          print("Validation Accuracy: %.1f%%" % accuracy(valid_preds[0], valid_labels))
  test_preds = session.run([tf_predictions], feed_dict={tf_X_input:test_dataset})
  print("Test accuracy: %.1f%%" % accuracy(test_preds[0], test_labels))

Initialized
Minibatch loss at step 0: 2.305990
Minibatch accuracy: 14.8%
Validation Accuracy: 46.5%
Minibatch loss at step 500: 0.421147
Minibatch accuracy: 89.1%
Validation Accuracy: 85.9%
Minibatch loss at step 1000: 0.536175
Minibatch accuracy: 85.2%
Validation Accuracy: 86.4%
Minibatch loss at step 1500: 0.442073
Minibatch accuracy: 88.3%
Validation Accuracy: 87.2%
Minibatch loss at step 2000: 0.295959
Minibatch accuracy: 92.2%
Validation Accuracy: 87.9%
Minibatch loss at step 2500: 0.332151
Minibatch accuracy: 88.3%
Validation Accuracy: 88.3%
Minibatch loss at step 3000: 0.266781
Minibatch accuracy: 93.8%
Validation Accuracy: 88.4%
Minibatch loss at step 3500: 0.308270
Minibatch accuracy: 92.2%
Validation Accuracy: 88.1%
Minibatch loss at step 4000: 0.221826
Minibatch accuracy: 94.5%
Validation Accuracy: 88.2%
Minibatch loss at step 4500: 0.307984
Minibatch accuracy: 91.4%
Validation Accuracy: 88.8%
Minibatch loss at step 5000: 0.416013
Minibatch accuracy: 89.8%
Validation Accurac

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

The overfitting case increases (~5%) when using dropout

In [72]:
batch_size = 128
reg_constant = .002
hidden_nodes_1 = 1024
hidden_nodes_2 = 512
learning_rate = 0.5
dropout_rate = 0.5

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_X_input = tf.placeholder(tf.float32,
                                    shape=(None, image_size * image_size))
  tf_y_input = tf.placeholder(tf.float32, shape=(None, num_labels))
  tf_is_training = tf.placeholder(tf.bool)
  
  # Variables.
  tf_layer_1 = tf.layers.dense(tf_X_input,hidden_nodes_1)
  tf_layer_1 = tf.nn.relu(tf_layer_1)
  tf_layer_1 = tf.layers.dropout(tf_layer_1,rate = dropout_rate, training = tf_is_training)
  tf_layer_2 = tf.layers.dense(tf_layer_1,hidden_nodes_2)
  tf_layer_2 = tf.nn.relu(tf_layer_2)
  tf_output = tf.layers.dense(tf_layer_2,num_labels)
  

  # Training computation.
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_y_input, logits=tf_output))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  
  # Predictions
  tf_predictions = tf.nn.softmax(tf_output)

In [73]:
num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (500)#Limited Batch Size
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_X_input : batch_data, tf_y_input : batch_labels, tf_is_training : True}
    _, l, predictions = session.run(
      [optimizer, loss, tf_predictions], feed_dict=feed_dict)
    if (step % 1000 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          valid_preds = session.run([tf_predictions], feed_dict={tf_X_input:valid_dataset, tf_is_training:False})
          print("Validation Accuracy: %.1f%%" % accuracy(valid_preds[0], valid_labels))
  test_preds = session.run([tf_predictions], feed_dict={tf_X_input:test_dataset, tf_is_training:False})
  print("Test accuracy: %.1f%%" % accuracy(test_preds[0], test_labels))

Initialized
Minibatch loss at step 0: 2.426312
Minibatch accuracy: 10.2%
Validation Accuracy: 34.9%
Minibatch loss at step 1000: 0.001213
Minibatch accuracy: 100.0%
Validation Accuracy: 78.7%
Minibatch loss at step 2000: 0.000284
Minibatch accuracy: 100.0%
Validation Accuracy: 79.0%
Minibatch loss at step 3000: 0.000206
Minibatch accuracy: 100.0%
Validation Accuracy: 78.9%
Minibatch loss at step 4000: 0.000403
Minibatch accuracy: 100.0%
Validation Accuracy: 79.0%
Minibatch loss at step 5000: 0.000083
Minibatch accuracy: 100.0%
Validation Accuracy: 79.1%
Test accuracy: 86.3%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [89]:
batch_size = 128
hidden_nodes = 1024
learning_rate = 0.8
dropout_rate = 0.5

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_X_input = tf.placeholder(tf.float32,
                                    shape=(None, image_size * image_size))
  tf_y_input = tf.placeholder(tf.float32, shape=(None, num_labels))
  tf_is_training = tf.placeholder(tf.bool)
  
  # Variables.
  tf_layer_1 = tf.layers.dense(tf_X_input,hidden_nodes)
  tf_layer_1 = tf.nn.relu(tf_layer_1)
  tf_layer_1 = tf.layers.dropout(tf_layer_1,rate = dropout_rate, training = tf_is_training)
  tf_layer_2 = tf.layers.dense(tf_layer_1,hidden_nodes)
  tf_layer_2 = tf.nn.relu(tf_layer_2)
  tf_layer_3 = tf.layers.dense(tf_layer_2,hidden_nodes)
  tf_layer_3 = tf.nn.relu(tf_layer_3)
  tf_output = tf.layers.dense(tf_layer_3,num_labels)
  

  # Training computation.
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_y_input, logits=tf_output))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  
  # Predictions
  tf_predictions = tf.nn.softmax(tf_output)

In [90]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_X_input : batch_data, tf_y_input : batch_labels, tf_is_training : True}
    _, l, predictions = session.run(
      [optimizer, loss, tf_predictions], feed_dict=feed_dict)
    if (step % 1000 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          valid_preds = session.run([tf_predictions], feed_dict={tf_X_input:valid_dataset, tf_is_training:False})
          print("Validation Accuracy: %.1f%%" % accuracy(valid_preds[0], valid_labels))
  test_preds = session.run([tf_predictions], feed_dict={tf_X_input:test_dataset, tf_is_training:False})
  print("Test accuracy: %.1f%%" % accuracy(test_preds[0], test_labels))
  winsound.Beep(500,1000)

Initialized
Minibatch loss at step 0: 2.374178
Minibatch accuracy: 8.6%
Validation Accuracy: 27.1%
Minibatch loss at step 1000: 0.564951
Minibatch accuracy: 82.8%
Validation Accuracy: 85.9%
Minibatch loss at step 2000: 0.380035
Minibatch accuracy: 88.3%
Validation Accuracy: 88.0%
Minibatch loss at step 3000: 0.368844
Minibatch accuracy: 89.1%
Validation Accuracy: 88.3%
Minibatch loss at step 4000: 0.269961
Minibatch accuracy: 93.8%
Validation Accuracy: 88.7%
Minibatch loss at step 5000: 0.442255
Minibatch accuracy: 85.9%
Validation Accuracy: 89.1%
Minibatch loss at step 6000: 0.451022
Minibatch accuracy: 85.9%
Validation Accuracy: 89.3%
Minibatch loss at step 7000: 0.367649
Minibatch accuracy: 89.1%
Validation Accuracy: 89.5%
Minibatch loss at step 8000: 0.399015
Minibatch accuracy: 86.7%
Validation Accuracy: 89.5%
Minibatch loss at step 9000: 0.316912
Minibatch accuracy: 91.4%
Validation Accuracy: 89.7%
Minibatch loss at step 10000: 0.232577
Minibatch accuracy: 92.2%
Validation Accura